（1）ortools结果转为自己的表示，类似于output里写的东西

（2）评估解（用一个类描述VRP问题）

（3）定义扰动

（4）模拟退火（或其他元启发式）

In [ ]:
solution_code=[[]]#[[15,3,8,0,6,5,11,4,0,...],...,[0,0,5,3,0,1,0,0]，用0分割不同车，按下标排序;最后一趟可能有很多0

In [ ]:
def exchangesolution():
    '''输入ortools的solution,返回我们定义的solution_code'''

In [ ]:
class VRP():
    def __init__(self,nodes,vehicles):
        self.nodes
        
        
    def evaluate(self,solution):
        '''
        输入solution_code,返回目标函数值，
        现在目标是总距离，返回总距离'''
        return total_dist
    
    def check_constraint(self,solution):
        '''输入solution_code,返回bool变量，判断解是否满足约束'''
        return True #/False
    
    def get_result(self,solution):
        '''输入solution_code,和其他需要的信息,返回满足json格式的dict变量，最后有空再写'''
        
        return result
        

In [ ]:
def ls_swap(solution):
    #交换
    return new_solution


In [ ]:
def sa(initials,tmin=0.05,t=35,alpha=0.94,beta):
    '''模拟退火，以前我写过，我找一下。
    initials:初始solution_code'''
    ini_fun = vrp.evaluate(initials)
    best_fun = ini_fun
    best_code = initials
    solution = initials
    while t > tmin:
        fun = vrp.evaluate(solution)
        if de > 0 & (np.random.rand() > np.exp(-de / (t * k))):
            solution=new_solution
            
        k = self.connect_matrix.sum()
        for i in range(5 + self.num_hidden_node):
            for j in range(self.num_hidden_node + 1):
                if self.connect_matrix[i, j] != 0:
                    weight = self.weight_matrix[i, j]
                    # self.weight_matrix[i, j] = self.weight_matrix[i, j] + random.randrange(-20, 20)
                    self.weight_matrix[i, j] = random.randrange(-100, 100)
                    new_error = self.get_error()
                    de = new_error - error
                    if de > 0 & (np.random.rand() > np.exp(-de / (t * k))):
                        self.weight_matrix[i, j] = weight
                    else:
                        error = new_error
                        if error < best_error:
                            best_error = error
                            best_weight = self.weight_matrix.copy()
                            best_bias = self.bias.copy()
                    k -= 1
        k = self.hidden_node.sum()+1
        for j in range(self.num_hidden_node):
            if self.hidden_node[j] != 0:
                bias = self.bias[j]
                # self.bias[j] += random.randrange(-20, 20)
                self.bias[j] = random.randrange(-100, 100)
                new_error = self.get_error()
                de = new_error - error
                if de > 0 & (np.random.rand() > np.exp(-de / t * k)):
                    self.bias[j] = bias
                else:
                    error = new_error
                    if error < best_error:
                        best_error = error
                        best_weight = self.weight_matrix.copy()
                        best_bias = self.bias.copy()
                k -= 1
        t = t * alpha
       # print(error)
    if error > best_error:
        self.weight_matrix = best_weight.copy()
        self.bias = best_bias.copy()
    if ini_error - best_error > 2:
        self.success = True
    else:
        self.success = False
    self.get_error()